## Proyecto final para Codigo Facilito Bootcamp

Este cuaderno contiene datos y hallazgos con respecto a la base datos de una compañia de taxis.


### ¿Que gestiona la app de taxis?

En un principio se buscaba que fuera un backend que pudiera otorgar comunicacion a tres aplicaciones:
- `Un sistema web basado en Vue`
    - Este sistema es conocido como Cabina en el cual por medio de roles y privilegios se dan de alta:
        - usuarios de cabina y/o administrativos.
        - los datos conductores como sus licencias con expiracion y una referencia fotografica de ellos mismos.
        - de los vehiculos se toman marca,modelo, placas, numeros economicos, permisos de transito y color.
        - Se pueden rastrear y dar de alta nuevos servicios.
- `La apliacion movil de usaurios (Android/iOS)`
    - Como una propuesta local para otorgar a usuarios mediante sana competencia una alternativa a Uber/Didi/Cabify.
- `La aplicacion movil de conductor (Android/iOS)`
    - Los conductores de taxis pueden otorgar servicio tradicional mediante paradas en la calle.
    - Cabina puede enviarles servicios para atenderlos con las direcciones exactas.
    - Dicha aplicacion sirve para monitorear (Casi en tiempo real) la geolocalizacion de la flotilla.
    - Dicha aplicacion tambien tiene accesos a Google maps y waze para servicios de GPS.

### Sobre los datos
Se me otorgo un respaldo completo de la base datos de Mysql la cual monte en mi maquina local, con dicha base datos montada he realizado algunas consultas las cuales converti en archivos `json` a manera de quitar la dependencia de una base datos y tener el motor,  asi tambien con el proposito de limpiar los datos de informacion sensible (Tokens de firebase notifiactions, passwords cifradas, correos, direcciones de correo, direcciones fisicas, nombres de personas, placas de vehiculos, numeros economicos,... etc)

Para lo cual en la carpeta de `data` de este proyecto estaran contenidos dichos `json`

### Librerias a usar
Para este proyecto se esta utilizando principalmente polars

In [2]:
%pip install pandas
%pip install nbformat 
%pip install polars
%pip install plotly

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

## Importaciones Globales de librerias

In [109]:
import polars as pl
import plotly.express as px
from datetime import datetime

### Exploracion de Datos

# Usuarios de la aplicacion
en la siguiente seccion se cargan los usuarios del sistema

In [3]:
usersDF = pl.read_json("data/GeneralUsersData.json")
usersDF

idUsuario,idRole,rol,status,created_at,updated_at
i64,i64,str,i64,str,str
1,1,"""Admin""",1,"""2020-02-20 07:…","""2020-02-20 07:…"
101,1,"""Admin""",1,"""2020-03-06 04:…","""2020-03-06 04:…"
174,2,"""Supervisor""",1,"""2020-04-09 14:…","""2020-04-09 14:…"
79,3,"""Operative""",1,"""2020-02-20 07:…","""2020-02-20 07:…"
2,4,"""Driver""",1,"""2020-02-20 07:…","""2021-03-03 07:…"
3,4,"""Driver""",1,"""2020-02-20 07:…","""2023-06-14 04:…"
5,4,"""Driver""",1,"""2020-02-20 07:…","""2020-02-20 07:…"
7,4,"""Driver""",1,"""2020-02-20 07:…","""2021-05-31 06:…"
9,4,"""Driver""",1,"""2020-02-20 07:…","""2020-02-20 07:…"


### ¿Que se puede decir de los usuarios?

De los usuarios del sistema se observa que existen `6` roles en el sistema
- Driver
- Client
- Supervisor
- Operative
- Admin

De los cuales predominan los usuarios de tipo `driver` y `client`
al mismo tiempo se hace notar que existen `4` usuarios de tipo extraordinario desactivados

In [73]:
filteredActiveUsersDF = usersDF.filter(pl.col('status') == 1)
resultActive = filteredActiveUsersDF.group_by('rol').agg(pl.count('rol').alias('cantidad'))
resultActive

usersFig = px.bar(resultActive, x="rol", y="cantidad", color="rol", title="Cantidad de usuarios a segun su rol")
usersFig.show()


In [57]:
filteredNonActiveUsersDF = usersDF.filter(pl.col('status') == 0)
resultNonActive = filteredNonActiveUsersDF.group_by('rol').agg(pl.count('rol').alias('conteo'))
resultNonActive

rol,conteo
str,u32
"""Extraordinario…",4


In [79]:
vehiclesDF = pl.read_json("data/GeneralVehiclesData.json")
vehiclesActive = vehiclesDF.filter(pl.col("activo") == 1)
vehiclesActive

idVehicle,marca,modelo,year,color,activo,expiracionDePermiso,idTipoVehiculo,tipoVehiculo
i64,str,str,i64,str,i64,str,i64,str
57,"""NISSAN""","""VERSA""",2017,"""#FFFBFB""",1,"""2020-04-15""",1,"""Regular"""
62,"""NISSAN""","""TSURU""",2013,"""#FFFFFF""",1,"""2020-10-20""",1,"""Regular"""
66,"""0000000""","""0000000000""",0,"""#FFFFFF""",1,"""2020-04-11""",1,"""Regular"""
67,"""VERSA""","""2018""",2018,"""#FFF""",1,"""2020-04-24""",1,"""Regular"""
68,"""NISSAN TSURU""","""2014""",2014,"""#FFF""",1,"""2021-01-20""",1,"""Regular"""
70,"""NISSAN""","""TSURU""",2017,"""#FFF""",1,"""2020-04-13""",1,"""Regular"""
72,"""NISSAN""","""TSURU""",2012,"""#FFF""",1,"""2020-04-13""",1,"""Regular"""
73,"""TSURU""","""NISSAN""",2012,"""#FFF""",1,"""2020-12-27""",1,"""Regular"""
74,"""NISSAN""","""VERSA""",2013,"""#FFF""",1,"""2020-04-14""",1,"""Regular"""


In [102]:
vehiclesActiveClean = vehiclesActive.with_columns(
    pl.col('marca')
    .str.replace('CHEVY', 'CHEVROLET')
    .str.replace('VERSA', 'CHEVROLET')
    .str.replace('CHEVROLET AVEO', 'CHEVROLET')
    .str.replace('CHEVRILET', 'CHEVROLET')
    .str.replace('VOLJSWAGEN', 'VOLKSWAGEN')
    .str.replace('Volkswagen', 'VOLKSWAGEN')
    .str.replace('VOLSKWAGEN', 'VOLKSWAGEN')
    .str.replace('VW', 'VOLKSWAGEN')
    .str.replace('NISSSAN', 'NISSAN')
    .str.replace('NISSAN TSURU', 'NISSAN')
    .str.replace('TSURU', 'NISSAN')
    .str.replace('DOGDE', 'DODGE')
    .str.replace(r'^0+$', 'NA')
    .str.replace(r'^Unidad Regular+$', 'NA')
    .str.replace(r'^S$', 'NA')
)
vehiclesActiveClean = vehiclesActiveClean.with_columns(
    pl.col('expiracionDePermiso').str.to_date('%Y-%m-%d').alias('expiracionDePermisoDate')
)
groupbyMarcas = vehiclesActiveClean.group_by('marca').agg(pl.count('marca').alias('cantidad'))
groupbyMarcas = groupbyMarcas.filter( pl.col("marca") != 'NA')
vehiclesBrandFig = px.bar(groupbyMarcas, x="marca", y="cantidad", color="marca", title="Principales marcas de autos en la flotilla")
vehiclesBrandFig.show()

In [103]:
groupbyTipoDeVehiculo = vehiclesActiveClean.group_by('tipoVehiculo').agg(pl.count('tipoVehiculo').alias('cantidad'))

vehiclesBrandFig = px.pie(groupbyTipoDeVehiculo, values="cantidad", names="tipoVehiculo", title="Relación entre tipos de vehiculos en la flotilla")

vehiclesBrandFig.show()

In [104]:
vehiclesActiveCleanYears = vehiclesActiveClean.with_columns(
    pl.col('year')
    .str.replace(r'^0+$', '2010')
    
)
groupbyYears = vehiclesActiveClean.group_by('year', 'marca').agg(pl.count('year').alias('cantidad')).sort('year')
groupbyYears = groupbyYears.filter( pl.col("marca") != 'NA')

vehiclesBrandFig = px.bar(groupbyYears, x="marca", y="cantidad", color='year', hover_name="year",
color_discrete_sequence=px.colors.qualitative.Antique, title="Cantidad de vehiculos por marca agrupados por año")

vehiclesBrandFig.show()

In [130]:
# vehiclesActiveClean
vehiculosConPermisoExpirado = vehiclesActiveClean.filter(pl.col("expiracionDePermisoDate") <= datetime(2023,9,24))
vehiculosConPermisoVigente = vehiclesActiveClean.filter(pl.col("expiracionDePermisoDate") >= datetime(2023,9,24))
cantidadDeVehiculos = {
  'permiso': ['Expirado', 'Vigente'],
  'cantidadPermisos': [vehiculosConPermisoExpirado.height,vehiculosConPermisoVigente.height]
}

# cantidadDeVehiculos
vehiclesBrandFig = px.pie(cantidadDeVehiculos, values="cantidadPermisos", names="permiso", title="Cantidad de vehiculos con permisos expirados contra vehiculos con permisos vigentes")

vehiclesBrandFig.show()